# __MLflow - Red Neuronal__

## __Generator v.0__

### __Objective__

- Develop an exercise with MLflow to learn about this tool.

Paginas para revisar:

- https://www.mlflow.org/
- https://learn.microsoft.com/en-us/azure/databricks/_static/notebooks/mlflow/mlflow-model-registry-example.html
- https://learn.microsoft.com/en-us/azure/databricks/mlflow/model-registry-example
- https://medium.com/@haythemtellili/end-to-end-ml-pipelines-with-mlflow-projects-63a11baa2dd1
- https://medium.com/@kevin.n.lu123/mlflow-managing-your-ml-pipeline-from-training-to-deployment-7e0d87df9d
- https://crunchingthedata.com/cs01-mlflow-models/
- https://medium.com/noodle-labs-the-future-of-ai/introduction-to-mlflow-for-mlops-part-1-anaconda-environment-1fd9e299226f
- https://medium.com/noodle-labs-the-future-of-ai/introduction-to-mlflow-for-mlops-part-2-docker-environment-53516ce45266
- https://medium.com/noodle-labs-the-future-of-ai/introduction-to-mlflow-for-mlops-part-3-database-tracking-minio-artifact-storage-and-registry-9fef196aaf42
- https://kili-technology.com/blog/how-to-manage-your-machine-learning-pipeline-with-mlflow
- https://analyticsindiamag.com/kubeflow-vs-mlflow-which-mlops-tool-should-you-use/

MLflow - Storing Artifacts in HDFS and in an SQLite DB: 

- https://medium.com/@moyukh_51433/mlflow-storing-artifacts-in-hdfs-and-in-an-sqlite-db-7be26971b6ab

Cursos udemy:
- https://www.udemy.com/course/azure-machine-learning-mlops-mg/

Youtube:

- https://www.youtube.com/watch?v=wb-ZxtIwSTA
- https://www.youtube.com/watch?v=JApPzAnbfPI
- https://www.youtube.com/watch?v=SCwi3b29qwY

In [ ]:
#!pip install mlflow
# !pip install pyyaml
#!pip install sqlparse
#!pip install #!pip installquerystring-parser
#!pip install flask
#!pip install waitress
#!pip install waitress threadpoolctl

In [ ]:
import numpy as np
import pandas as pd
import warnings
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
warnings.filterwarnings("ignore")

In [ ]:
import mlflow
import mlflow.keras
mlflow.sklearn.autolog()

In [ ]:
import sys
#sys.argv
print(sys.argv, len(sys.argv))

In [ ]:
import mlflow
from pathlib import Path

# Create an experiment name, which must be unique and case sensitive
experiment_id_tf = mlflow.create_experiment(
    "TF Experiments",
    artifact_location=Path.cwd().joinpath("tutorial_mlflow/mlflow-tensorflow").as_uri(),
    tags={"version": "v1", "priority": "P1"},
)

In [ ]:
experiment_id_tf = "0"

In [ ]:
experiment = mlflow.get_experiment(experiment_id_tf)
print("Name: {}".format(experiment.name))
print("Experiment_id: {}".format(experiment.experiment_id))
print("Artifact Location: {}".format(experiment.artifact_location))
print("Tags: {}".format(experiment.tags))
print("Lifecycle_stage: {}".format(experiment.lifecycle_stage))
#print("Creation timestamp: {}".format(experiment.creation_time))

In [ ]:
# Enviar experimento a la carpeta de "trash"
# Fuente: https://mlflow.org/docs/latest/python_api/mlflow.html#mlflow.delete_experiment
import mlflow

experiment_id = mlflow.create_experiment("Experiment_NEW")
#experiment_id = '0'
mlflow.delete_experiment(experiment_id)

# Examine the deleted experiment details.
experiment = mlflow.get_experiment(experiment_id)
print("Name: {}".format(experiment.name))
print("Artifact Location: {}".format(experiment.artifact_location))
print("Lifecycle_stage: {}".format(experiment.lifecycle_stage))
#print("Last Updated timestamp: {}".format(experiment.last_update_time))

Tambien se tiene que eliminar de la base datos

* DELETE FROM experiments WHERE experiment_id = 1;
* DELETE FROM experiment_tags WHERE experiment_id = 1;

# Ejemplo de Red Neuronal

In [ ]:
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt

X,y = make_blobs(n_samples=1000, n_features=2, centers=2, random_state=42)

plt.scatter(X[:,0], X[:, 1], c=y, marker="o")
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
len(X_train.tolist())

In [ ]:
from mlflow.tracking import MlflowClient
from mlflow.models.signature import infer_signature
client = MlflowClient()

In [ ]:
import os
import warnings
import sys
import pandas as pd
import numpy as np
#import tensorflow
from tensorflow import keras
from tensorflow.keras.datasets import reuters
from tensorflow.keras.optimizers import *
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.preprocessing.text import Tokenizer
from urllib.parse import urlparse
import mlflow
import mlflow.tensorflow

In [ ]:
def print_auto_logged_info(r):
    tags = {k: v for k, v in r.data.tags.items() if not k.startswith("mlflow.")}
    artifacts = [f.path for f in MlflowClient().list_artifacts(r.info.run_id, "model")]
    print("run_id: {}".format(r.info.run_id))
    print("artifacts: {}".format(artifacts))
    print("params: {}".format(r.data.params))
    print("metrics: {}".format(r.data.metrics))
    print("tags: {}".format(tags))

def train_keras_model(X, y,learning_rates,optimizer,epochs,batch_size,validation_split):
  optimizer=keras.optimizers.Adam(learning_rate=learning_rates)

  model_anr = Sequential()
  model_anr.add(Dense(8, input_shape=(X_train.shape[-1],), activation="relu", name="hidden_layer"))
  model_anr.add(Dense(1,activation = "sigmoid"))
  model_anr.compile(loss="binary_crossentropy", optimizer=optimizer, metrics=["binary_accuracy"])

  history = model_anr.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=validation_split)

  fig, ax = plt.subplots(2,1)
  ax[0].plot(history.history['loss'], color='b', label="Training loss")
  ax[0].plot(history.history['val_loss'], color='r', label="Validation loss",axes =ax[0])
  legend = ax[0].legend(loc='best', shadow=True)

  ax[1].plot(history.history['binary_accuracy'], color='b', label="Training binary accuracy")
  ax[1].plot(history.history['val_binary_accuracy'], color='r',label="Validation binary accuracy")
  legend = ax[1].legend(loc='best', shadow=True)

  test_loss, test_binary_accuracy = model_anr.evaluate(X_test, y_test)
  print("Test test_binary_accuracy: {}".format(test_binary_accuracy))

  mlflow.log_metric("test_loss", test_loss)
  mlflow.log_metric("test_binary_accuracy", test_binary_accuracy)
  return model_anr, history


In [ ]:
experiment_id_tf = '0'

In [ ]:
num_experiment = "Experiment 5"

if __name__ == "__main__":
    mlflow.set_tracking_uri("http://localhost:5000")
    warnings.filterwarnings("ignore")
    np.random.seed(40)
    
    mlflow.keras.autolog()
    #mlflow.autolog()
    #with mlflow.start_run(experiment_id=experiment_id_tf) as run:
    with mlflow.start_run(experiment_id=experiment_id_tf, run_name=num_experiment) as run:

          # Automatically capture the model's parameters, metrics, artifacts,
            # and source code with the `autolog()` function
            #mlflow.keras.autolog()

            mlflow.set_tag("mlflow.runName", num_experiment)

            learning_rates = 0.001
            optimizer=keras.optimizers.Adam(learning_rate=learning_rates)
            epochs = 10
            batch_size = 32
            validation_split = 0.1

            model_anr, history  = train_keras_model(X, y,learning_rates,optimizer,epochs,batch_size,validation_split)

            # get model signature
            signature = infer_signature(model_input=X_train, model_output=model_anr.predict(X_train))

            # Save the plot and log it as an artifact
            
            plt.savefig("behavior_plot.png")
            mlflow.log_artifact("behavior_plot.png")

            tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

            print('tracking_url_type_store:',tracking_url_type_store)

            #from tensorflow.python.saved_model import signature_constants
            #tag=[keras.saved_model.tag_constants.SERVING]
            #tag=[tensorflow.compat.v1.saved_model.tag_constants.SERVING]
            #key=signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY

            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                # Save model to artifacts
                mlflow.keras.log_model(keras_model = model_anr, artifact_path='keras-models',signature=signature,
                registered_model_name="keras-modelo-1")
            else:
                # Save model to artifacts
                mlflow.keras.log_model(keras_model = model_anr, artifact_path="keras-models",signature=signature,
                registered_model_name="keras-modelo-1")
                #mlflow.keras.log_model()

# fetch the auto logged parameters and metrics for ended run
print_auto_logged_info(mlflow.get_run(run_id=run.info.run_id))

mlflow.end_run()
